In [1]:
import os 
import numpy as np 
import pickle
from matplotlib import pyplot as plt
import matplotlib
import shutil
# import seaborn as sns
# from traj_run import rho_ohmic
from typing import Any
from numpy import dtype, ndarray
import scipy
from scipy.optimize import curve_fit
from scipy.fft import fft, fftfreq
from scipy.interpolate import interp1d
import pywt
from data_analysis import read_line, read_exp, read_omega, get_rho_array, translate_param, chunk_data, interp_dat, do_fft, \
    fft_analysis, func_gentor, draw_t_S, draw_w_S, wavelet_denoising, line_monotonic_detect, show_result_t_S, \
    show_fft_res, show_w_freqs, show_w_freqs1, get_data_of_vodf, get_signal_freq, do_cft

2024-09-23 14:52:42,989[INFO] Use NumPy as backend
2024-09-23 14:52:42,990[INFO] numpy random seed is 9012
2024-09-23 14:52:42,990[INFO] random seed is 1092
2024-09-23 14:52:42,992[INFO] Git Commit Hash: 0e92cabebc362b15d1335e570090315033d407d5
2024-09-23 14:52:42,992[INFO] use 64 bits


In [4]:

mother_folder = '/home/ansatz/data/comp/sbm/24_09_15'

In [ ]:
vn_entropy_1site_all_lines ={}
# for line in sorted([ s for s in os.listdir(mother_folder) if s.startswith('traj_s') ]):
#     points = { i_step : f"_{i_step}_step_entropy_1sites.pickle" for i_step in range(100) }
#     nmodes = int(os.path.basename(line).split('_nmodes')[1].split('_')[0])

#     rho_type = 0 if 'rho_type' not in os.path.basename(line) else int(os.path.basename(line).split('rho_type_')[1])    
#     key = f"{line.split('_')[1]}-{line.split('_')[2]}-nmodes{nmodes}-rho{rho_type}"
#     print(key)
#     vn_entropy_1site_all_lines[key] = points
print('-----')
for line in sorted([ s for s in os.listdir(mother_folder) if s.startswith('traj_s') ]):
    points = { i_step : f'{i_step:04}_step_entropy_1site.pickle' for i_step in range(100) }
    nmodes = int(os.path.basename(line).split('_nmodes')[1].split('_')[0])

    rho_type = 0 if 'rho_type' not in os.path.basename(line) else int(os.path.basename(line).split('rho_type_')[1])
    key = f"{line.split('_')[1]}-{line.split('_')[2]}-nmodes{nmodes}-rho{rho_type}"
    # print(key)
    vn_entropy_1site_all_lines[key] = points

In [ ]:
vn_entropy_1site_all_lines.keys()

In [7]:
def get_freq_cft_xf_yf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100,
                  idof=504):
    v_dof = data = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=s, alpha=alpha, nmodes=nmodes, rho_type=rho_type, nsteps=nsteps,
                  idof=idof)
    signal = wavelet_denoising(v_dof)
    N = 1001
    xf, yf = do_cft(np.linspace(0, 10, 100), signal, N)

    plt.clf()

    
    amp = 2.0/N * np.abs(yf)

    plt.title(f's_{s}-alpha_{alpha}-nmodes_{nmodes}-rho_type_{rho_type}-nsteps_{nsteps}')
    plt.plot(xf, amp, label=f'v_{idof}')
    indexs , _ = scipy.signal.find_peaks(amp)
    
    if len(indexs) != 0 : 
        peaks = [ amp[index] for index in indexs ]
        freqs = [ xf[index] for index in indexs ]
        with open(f'data/s{s}-alpha{alpha}-idof_{idof:03}_cft_peaks.pickle', 'wb') as f:
            pickle.dump([freqs, peaks], f)
        # index = indexs[peaks.index(max(peaks))]
        # index = indexs[0]
        # print(peaks.index(max(peaks)), index)
        # index = indexs[0]
        # freq =  xf[index]
        # amplitude = amp[index]
        # amplitude = 2.0/N * np.abs(yf[index])
        # phase = np.angle(np.abs(yf[index]))
        # print(freq, amplitude)
        sorted_peaks = sorted(peaks, reverse=True)
        index = indexs[peaks.index(sorted_peaks[0])]
        freq =  xf[index]
        amplitude = amp[index]
        plt.scatter(freq, amplitude, color='red')
        plt.annotate(text=f'{xf[index]}_{amp[index]}', xy=(xf[index], amp[index]), xytext=(xf[index], amp[index]))
        if len(indexs) >= 1:
            index = indexs[peaks.index(sorted_peaks[1])]
            freq =  xf[index]
            amplitude = amp[index]
            plt.scatter(freq, amplitude, color='red')
            plt.annotate(text=f'{xf[index]}_{amp[index]}', xy=(xf[index], amp[index]), xytext=(xf[index], amp[index]))

        if len(indexs) >= 2:
            index = indexs[peaks.index(sorted_peaks[2])]
            freq =  xf[index]
            amplitude = amp[index]
            plt.scatter(freq, amplitude, color='red')
            plt.annotate(text=f'{xf[index]}_{amp[index]}', xy=(xf[index], amp[index]), xytext=(xf[index], amp[index]))
        plt.legend()
        plt.savefig(f'data/s{s}-alpha{alpha}-idof_{idof:03}_cft_peaks.png')
        # plt.clf()
        return freq
    else:
        return 0


In [8]:
ids = [ i for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
dats = [get_signal_freq(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=200, imode=i, plot=False) for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
plt.plot([ _[0] for _ in dats ], [ _[1] for _ in dats ])
plt.xlim(0, 10)

In [11]:
dats = [get_signal_freq(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.8, nmodes=1000, rho_type=0, nsteps=100, imode=i, plot=False) for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
plt.plot([ _[0] for _ in dats ], [ _[1] for _ in dats ])
plt.xlim(0, 10)

In [13]:
dats = [get_signal_freq(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.05, nmodes=1000, rho_type=0, nsteps=200, imode=i, plot=False) for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
plt.plot([ _[0] for _ in dats ], [ _[1] for _ in dats ])
plt.xlim(0, 10)

In [ ]:
dats = [get_freq_cft_xf_yf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=200, idof=i) for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
plt.plot(dats)

In [19]:
data_559 = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100,
                  idof=559)

In [20]:
data_564 = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100,
                  idof=564)

In [21]:
data_569 = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100,
                  idof=569)

In [ ]:
plt.plot(np.linspace(0,10,100), data_559, label='v559')
plt.plot(np.linspace(0,10,100), data_564, label='v564')
plt.plot(np.linspace(0,10,100), data_569, label='v569')
plt.legend()
plt.title('s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100')

In [7]:
ids = [ i for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [5]:
data = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100,
                  idof=504)

In [9]:
xf, yf = do_cft(np.linspace(0, 10, 100), v_dof, 1001)

In [7]:
v_dof = data = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines,\
                  s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100,
                  idof=564)
N = 1001
xf, yf = do_cft(np.linspace(0, 10, 100), v_dof, N)

In [6]:
def get_true_peaks_cft(xf, yf, N):
    
    amp = 2.0/N * np.abs(yf)

    max_indexs = scipy.signal.argrelmax(amp)[0]
    min_indexs = scipy.signal.argrelmin(amp)[0]
    peaks = []
    for i_peak in range(len(max_indexs)):
        peak_index = max_indexs[i_peak]
        left_index = min_indexs[min_indexs < peak_index][-1]
        right_index = min_indexs[min_indexs > peak_index][0]
        base_line = (amp[left_index] + amp[right_index])/2
        value = amp[peak_index] - base_line
        peaks.append(value)
    return max_indexs.tolist(), peaks


In [ ]:
get_true_peaks_cft(xf, yf, N)

In [ ]:
amp = 2.0/N * np.abs(yf)

max_indexs = scipy.signal.argrelmax(amp)[0]
min_indexs = scipy.signal.argrelmin(amp)[0]
peaks = []
for i_peak in range(len(max_indexs)):
    peak_index = max_indexs[i_peak]
    left_index = min_indexs[min_indexs < peak_index][-1]
    right_index = min_indexs[min_indexs > peak_index][0]
    base_line = (amp[left_index] + amp[right_index])/2
    value = amp[peak_index] - base_line
    peaks.append(value)

sorted_peaks = sorted(peaks, reverse=True)
index = max_indexs[peaks.index(sorted_peaks[0])] # xf index
freq =  xf[index]
amplitude = peaks[peaks.index(sorted_peaks[0])]
plt.scatter(freq, amplitude, color='red')
plt.annotate(text=f'{freq}_{amplitude:02f}', xy=(xf[index], amp[index]), xytext=(xf[index], amp[index]))
plt.plot(xf, amp)

if len(max_indexs) >= 1:
    index = max_indexs[peaks.index(sorted_peaks[1])]
    freq =  xf[index]
    amplitude = peaks[peaks.index(sorted_peaks[1])]
    plt.scatter(freq, amplitude, color='red')
    plt.annotate(text=f'{freq}_{amplitude:02f}', xy=(xf[index], amp[index]), xytext=(xf[index], amp[index]))
if len(max_indexs) >= 2:
    index = max_indexs[peaks.index(sorted_peaks[2])]
    freq =  xf[index]
    amplitude = peaks[peaks.index(sorted_peaks[2])]
    plt.scatter(freq, amplitude, color='red')
    plt.annotate(text=f'{freq}_{amplitude:02f}', xy=(xf[index], amp[index]), xytext=(xf[index], amp[index]))
plt.xlim(0.1, 5)

In [37]:
peak_index = max_indexs[0]
left_index = min_indexs[min_indexs < peak_index][-1]
right_index = min_indexs[min_indexs > peak_index][0]

In [ ]:
base_line = (amp[left_index] + amp[right_index])/2
value = amp[peak_index] - base_line

In [28]:
peak_index = max_indexs[0]

In [ ]:
min_indexs[min_indexs < peak_index][-1]

In [ ]:
min_indexs[min_indexs > peak_index][0]

In [ ]:
plt.plot(xf, np.abs(yf))

In [ ]:
imodes, ws, freqs, amps, xfs, fft_amps, key = show_result_t_S(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.4, nmodes=1000, rho_type=0, step_length=1, query_modes=None)

In [ ]:
show_w_freqs1(key, ws, freqs, imodes, xfs, fft_amps)

In [ ]:
# index = 40
# x, y =  xfs[index], fft_amps[index]
# plt.plot(x[:20], y[:20])

In [8]:
dats = [(get_signal_freq(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.4, nmodes=1000, rho_type=0, nsteps=100, imode=i, plot=False) , i )for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
dats

In [ ]:
vids[98]

In [ ]:
freqs[589]

In [ ]:
plt.plot(freqs)

In [ ]:

for i in range(0, 1000, 4):
    print(i)

In [ ]:
plt.title(f's0.7 alpha 0.4 nmodes 1000 rho_type 0 w-freqs from mode 99-629')
plt.plot(ws[imodes.index(94):imodes.index(629)], freqs,'o')
plt.xlabel('freq of phonon')
plt.ylabel('freq of S')
plt.xlim(0,10)
plt.ylim(0,max(freqs))

In [11]:
x = np.linspace(0, 10, 100)


In [12]:
signal = get_data_of_vodf(mother_folder=mother_folder, data_dict=vn_entropy_1site_all_lines, s=0.7, alpha=0.40, nmodes=1000, rho_type=0, idof=4, nsteps = 100)
x_uniform, y_uniform = interp_dat(np.linspace(0,10,100), np.array(signal), 1000, kind='quadratic')
y_deno = wavelet_denoising(y_uniform)

In [13]:
x_uniform, y_uniform = interp_dat(np.linspace(0,10,100), np.array(signal), 1000, kind='quadratic')

In [14]:
detect_res = line_monotonic_detect(y_uniform)

In [ ]:
plt.plot(x_uniform, y_uniform,label='int')
# plt.plot(x, signal,label='ori')
plt.legend()

In [16]:
y_deno = wavelet_denoising(y_uniform)

In [ ]:
plt.plot(np.linspace(0, 10, 1000), y_deno)

In [18]:

time_data= np.linspace(0, 10, 1000)

dt_indexs = scipy.signal.argrelmax(y_deno)[0]

In [19]:
def average_T(time_data, dt_indexs):
    Ts = []
    for i in range(1, len(dt_indexs)):
        index0 = dt_indexs[i-1]
        index1 = dt_indexs[i]
        T = time_data[index1] - time_data[index0]
        Ts.append(T)
    return np.mean(Ts)

In [ ]:
1/ average_T(time_data, dt_indexs)

In [16]:
xf, yf = do_fft(np.linspace(0, 10, 1000), y_deno, 1000)

In [17]:
N = 1000

In [18]:
freq, amplitude, phase = fft_analysis(xf, yf, N)

In [ ]:
plt.plot(xf[:100], 2.0/N * np.real(yf)[:100])

In [20]:
def test_1(imode):
    signal = get_data_of_vodf(s=0.7, alpha=0.40, nmodes=1000, rho_type=0, idof=imode, nsteps = 100)
    x_uniform, y_uniform = interp_dat(np.linspace(0,10,100), np.array(signal), 1000, kind='quadratic')
    y_deno = wavelet_denoising(y_uniform)
    plt.plot(y_deno, label=f'v_{imode}')
    N = 1000
    xf, yf = do_fft(np.linspace(0, 10, N), y_deno, N)
    freq, amplitude, phase = fft_analysis(xf, yf, N)
    plt.legend(loc=2, bbox_to_anchor=(1.0, 1.0))
    return freq

In [36]:
def test_2(imode):
    signal = get_data_of_vodf(s=0.7, alpha=0.40, nmodes=1000, rho_type=0, idof=imode, nsteps = 100)
    x_uniform, y_uniform = interp_dat(np.linspace(0,10,100), np.array(signal), 1000, kind='quadratic')
    y_deno = wavelet_denoising(y_uniform)

    time_data= np.linspace(0, 10, 1000)

    dt_indexs = scipy.signal.argrelmax(y_deno)[0]
    plt.plot(y_deno, label=f'v_{imode}')
    # N = 1000
    # xf, yf = do_fft(np.linspace(0, 10, N), y_deno, N)
    # freq, amplitude, phase = fft_analysis(xf, yf, N)
    freq = 1/ average_T(time_data, dt_indexs)
    plt.legend(loc=2, bbox_to_anchor=(1.0, 1.0))
    return freq

In [ ]:
freqs = [test_1(i) for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]



In [ ]:
freqs = [test_2(i) for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630]

In [ ]:
plt.plot([i for i in range(1000) if (i+1)%5==0 and i > 98 and i < 630], freqs)

In [ ]:
freqs

In [ ]:
freqs

In [ ]:
output.shape

In [ ]:
output = [ output[i][0] for i in range(len(output))]
plt.plot(output)

In [ ]:
scipy.signal.find_peaks(output)

In [ ]:
for i in range(3,10):
    print(i)